# 1.6 高阶函数

我们对强大的编程语言提出的要求之一就是能够通过将名称分配给通用模板（general patterns）来构建抽象，然后直接使用该名称进行工作。

为了将某些通用模板表达为具名概念（name concepts），我们需要构造一种“可以接收其他函数作为参数”或“可以把函数当做返回值”的函数。这种可以操作函数的函数就叫做**高阶函数**（higher-order functions）

## 1.6.1 作为参数的函数

思考以下三个计算求和的函数。第一个sum_naturals会计算从1到n的自然数之和：

In [1]:
def sum_naturals(n):
    total , k = 0 , 1
    while k <= n:
        total , k = total + k , k + 1
    return total

sum_naturals(100)

5050

第二个sum_cubes函数会计算1到n的自然数的立方之和

In [2]:
def sum_cubes(n):
    total , k = 0 , 1
    while k <= n:
        total , k = total + k*k*k , k+1
    return total

sum_cubes(100)

25502500

第三个pi_sum会计算某个式子，它的值会非常缓慢地收敛到π

In [3]:
def pi_sum(n):
    total , k = 0 ,1
    while k <= n:
        total , k = total + 8/((4*k-3)*(4*k-1)) , k+1
    return total

pi_sum(100)

3.1365926848388144

这三个函数在很大程度上是相同的，仅在名称和用于计算被加项k的函数上有所不同。我们可以通过在同一模板中填充槽位（slots）来生成每个函数：

In [4]:
# def <name>(n):
#    total ,k = 0,1
#    while k <= n:
#        total,k = total + <term>(k),k+1
#    return total

我们可以用上面的通用模板，将“槽位”转换为形式参数

In [5]:
def summation(n,term):
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k+1
    return total

def cube(x):
    return x*x*x

def sum_cubes(n):
    return summation(n,cube)

result = sum_cubes(3)

使用会返回其参数的identity函数，我们还可以使用完全相同的summation函数对自然数求和

In [6]:
def summation(n,term):
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k+1
    return total

def identity(x):
    return x

def sum_naturals(n):
    return summation(n,identity)

sum_naturals(10)

55

In [7]:
def pi_term(x):
    return 8 / ((4*x-3) * (4*x-1))

def pi_sum(n):
    return summation(n, pi_term)

pi_sum(1e6)

3.141592153589902

## 1.6.2 作为通用方法的函数

有了高阶函数后，会见到一种更加强大的抽象：用一些函数来计算表达的通用方法(general methods)，而且和它们调用的特定函数无关。

思考下面的例子，它实现了迭代改进(iterative improvement)的通用方法，并使用它来计算黄金比例。黄金比例通常被称为"phi"，是一个接近1.6的数字，经常出现在自然、艺术和建筑中。

迭代改进算法从方程的guess解（推测值）开始，重复应用update函数来改进该猜测，并调用close比较来检查当前的guess是否已经“足够接近”正确值。

In [8]:
def improve(update,close,guess=1):
    while not close(guess):
      guess = update(guess)
    return guess

这个improve函数是迭代求精(repetitive refinement)的通用表达式。它并不会指定要解决的问题，而是会将这些细节留给作为参数传入的update和close函数

黄金比例的一个著名特性是它可以通过反复叠加任何正数的倒数加上1来计算，而且它比它的平方小1。根据这些特性表示为与improve一起使用的函数

In [9]:
def approx_eq(x,y,tolerance=1e-15):
    return abs(x-y) < tolerance

In [10]:
def golden_update(guess):
    return 1/guess + 1

def square_close_to_successor(guess):
    return approx_eq(guess*guess,guess+1)

使用参数 golden_update 和 square_close_to_successor 来调用 improve 将会计算出黄金比例的有限近似值。

In [11]:
improve(golden_update,square_close_to_successor)

1.6180339887498951

该例子说明了计算机科学中的两个相关的重要思想：
- 命名和函数使我们能将大量的复杂事物进行抽象
- 正是由于我们对 Python 语言有一个极其通用的求解过程，小的组件才能组合成复杂的程序

接下来对新通用方法improve进行测试来检查其正确性，用黄金比例的精确闭式解与我们的迭代结果进行比较

In [12]:
from math import sqrt
phi = 1/2 + sqrt(5)/2
def improve_test():
    approx_phi = improve(golden_update,square_close_to_successor)
    assert approx_eq(phi,approx_phi),"phi differs from its approximation"

improve_test()

## 1.6.3 嵌套定义

思考问题：计算一个数的平方根。重复应用以下更新，x的值会收敛为a的平方根

In [13]:
def average(x,y):
    return (x+y)/2
def sqrt_upadate(x,a):
    return average(x,a/x)

上述代码存在两个问题：
- improve函数中update函数只接受一个参数guess，两个参数的更新函数与其不兼容
- 该代码指提供一次更新

解决方案：通过嵌套定义函数

In [14]:
def sqrt(a):
    def sqrt_update(x):
        return average(x,a/x)
    def sqrt_close(x):
        return approx_eq(x*x,a)
    return improve(sqrt_update,sqrt_close)

与局部赋值一样，局部的def语句只影响局部帧。被定义的函数仅在求解sqrt时在作用域内。

**词法作用域**：局部定义的函数也可以访问整个定义作用域内的名称绑定。在此示例中，sqrt_update引用名称a，它是封闭函数sqrt的形参。这种在嵌套定义之间共享名称的规则称为语法作用域。内部函数可以访问定义它们的环境中的名称（而不是它们被调用的位置）

我们需要两个扩展来使环境模型启用词法作用域：
- 1.每个用户定义的函数都有一个父环境：定义它的环境
- 2.调用用户定义的函数时，其局部帧会继承其父环境

在调用sqrt之前，所有函数都是在全局环境中定义的，因此都有相同的父级：全局环境。

相比之下，当Python计算sqrt的前两个子句时，它会创建局部换机关联的函数

In [15]:
sqrt(256)

16.0

**继承环境**：一个环境可以由任意长的帧链构成，并且总是以全局帧结束。通过使用嵌套的def语句来调用在其他函数中定义的函数，我们可以创建更长的（帧）链。调用sqrt_update的环境由三个帧组成：
- 局部帧sqrt_update
- 定义sqrt_update的sqrt帧(标记为f1)
- 全局帧

Python中词法作用域的两个关键优势：
- 局部函数的名称不会影响定义它的函数的外部名称，因为局部函数的名称将绑定在定义它的当前局部环境中，而不是全局环境中
- 局部函数可以访问外层函数的环境，这是因为局部环境的函数体求值环境会继承定义它的求值环境

这里的sqrt_update函数自带了一些数据：a在定义它的环境中引用的值，因为它以这种方式“封装”信息，所以局部定义的函数通常被称为闭包

## 1.6.4 作为返回值的函数

通过创建“返回值就是函数”的函数，我们可以在我们的程序中实现更强大的表达能力。带有词法作用域的编程语言的一个重要特性就是，局部定义函数在它们返回时仍旧持有所关联的环境

## 1.6.5 示例：牛顿法

牛顿法是一种经典的迭代方法，用于寻找函数的零点，其原理在《数值计算方法》中也有记载

In [16]:
def newton_update(f,df):
    def update(x):
        return x - f(x) / df(x)
    return update

In [17]:
def find_zero(f,df):
    def near_zero(x):
        return approx_eq(f(x),0)
    return improve(newton_update(f,df),near_zero)

举例：假设函数$f(x)=x^2 - a$的导数是线性方程$df(x) = 2x$

In [18]:
def square_root_newton(a):
    def f(x):
        return x*x-a
    def df(x):
        return 2*x
    return find_zero(f,df)

square_root_newton(64)

8.0

推广至n次方根，如下

In [24]:
def power(x,n):
    """返回x*x*...*x，n个x相乘"""
    product,k = 1,0
    while k < n:
        product , k = product * x,k+1
    return product

def nth_root_of_a(n,a):
    def f(x):
        return power(x,n) - a
    def df(x):
        return n*power(x,n-1)
    return find_zero(f,df)

# nth_root_of_a(2,64)
# nth_root_of_a(3,64)
nth_root_of_a(6,64)

2.0

所有计算中的近似误差都可以通过将approx_eq中的误差tolerance改为更小的数字来减小

需要注意的是，牛顿法并不总是收敛的，此事在《数值计算方法》中已有记载，也有其他的解决方法

## 1.6.6 柯里化

我们可以使用高阶函数将一个接受多个参数的函数转换为一个函数链，每个函数接受一个参数。

具体而言：给定一个函数$f(x,y)$，我们可以定义另一个函数$g$使得$g(x)(y)$等价于$f(x,y)$。在这里，$g$是一个该高阶函数，它接受单个参数x并返回另一个接受单个参数y的函数。这种转换称为柯里化(Curring)

【示例】：以下是pow函数的柯里化版本

In [25]:
def curried_pow(x):
    def h(y):
        return pow(x,y)
    return h
curried_pow(2)(3) # 这里curried_pow(x)返回一个函数h(y)，h(y)再接受y并计算结果

8

在Python这种通用的语言中，当我们需要一个只接受单个参数额度函数时，柯里化很有用。

例如，map模式就可以将但参数函数应用于一串值

【示例】：函数实现map模式

In [26]:
def map_to_range(start,end,f):
    while start < end:
        print(f(start))
        start = start+1

这样就可以使用map_to_range和curried_pow来计算2的前十次方，而不是专门编写一个函数

In [27]:
map_to_range(0,10,curried_pow(2))

1
2
4
8
16
32
64
128
256
512


在上述例子中，我们手动对pow函数进行了柯里化变换，得到了curried_pow。相反，我们可以定义函数来自动进行柯里化，以及逆柯里化变换

In [30]:
def curry2(f):
    """返回给定的双参数函数的柯里化版本"""
    def g(x):
        def h(y):
            return f(x,y)
        return h
    return g

def uncurry2(g):
    """返回给定的柯里化函数的双参数版本"""
    def f(x,y):
        return g(x)(y)
    return f

pow_curried = curry2(pow)
pow_curried(2)(5)

map_to_range(0,10,pow_curried(2))

1
2
4
8
16
32
64
128
256
512


curry2函数接受一个双参数函数f并返回一个单参数函数g。当g应用于参数x时，它返回一个单参数函数h。当h应用于参数y时，它调用f(x,y)。因此，curry2f(x)(y)等价于f(x,y)。uncurry2函数反转了柯里化变换，因此uncurry2(curry2(f))等价于f

In [31]:
uncurry2(pow_curried)(2,5)

32

## 1.6.7 Lambda表达式

对于一些临时的表达式，我们不需要将表达式的值与名称相关联。在Python中，我们可以使用lambda表达式临时创建函数，这些表达式会计算为未命名的函数。一个lambda表达式的计算结果是一个函数，它仅有一个返回表达式组委主体。不逊于使用赋值和控制语句

In [ ]:
def compose1(f,g):
    return lambda x : f(g(x))
# lambda            x         :            f(g(x))
# "A function that  takes x   and returns  f(g(x))"

lambda表达式的结果称为lambda函数（匿名函数）。它没有固定名称（因此Python打印<lambda>作为名称），但初次之外它的行为与任何其他函数都相同

In [33]:
s = lambda x : x*x
# s
s(12)

144

尽管lambda表达式很简洁，但是难辨认，如下

In [ ]:
compose1 = lambda f,g: lambda x: f(g(x))

## 1.6.8 抽象和一等函数

高等函数的重要性在于，它们使我们能够将这些抽象显式地表达为我们编程语言中的元素，以便可以像其他计算元素一样处理

一般而言，编程语言会对计算元素的操作方式施加限制。拥有最少限制的元素可以获得一等地位（first-class status）。这些一等元素的“权利和特权”包括：
- 1.可以与名称绑定
- 2.可以作为参数传递给函数
- 3.可以作为函数的结果返回
- 4.可以包含在数据结构中

Python 授予函数完全的一等地位，由此带来的表达能力的提升是巨大的。

## 1.6.9 函数装饰器

Python提供了一种特殊的语法来使用高阶函数作为执行def语句的一部分，称为装饰器（decorator）。最常见的例子是trace：

In [43]:
def trace(fn):
    def wrapped(x):
        print('->',fn,'(',x,')')
        return fn(x)
    return wrapped

@trace
def triple(x):
    return 3*x

triple(12)

-> <function triple at 0x00000221518E7B80> ( 12 )


36

在这个例子中，定义了一个高阶函数trace，它返回一个函数，该函数在调用其参数前显输出一个打印语句来显示该参数。tripe的def语句有一个注解（annotation）@trace，它会影响def执行的规则。和往常一样，函数triple被创建了。但是，名称triple不会绑定到这个函数上。相反，这个函数名称会被绑定在新定义的truple函数调用trace后返回的函数值上。上述装饰器等价于：

In [ ]:
def triple(x):
    return 3*x
triple = trace(triple)

在该笔记所对应教材相关项目中，装饰器被用于追踪，以及在从命令行运行程序时选择要调用哪些函数

补充：装饰器符号@也可以在后面跟一个调用表达式。跟在@后面的表达式会先辈解析（就像上面的'trace'名称一样），然后是def语句，最后讲装饰器表达式的运算结果应用到新定义的函数上，并将其结果绑定到def语句中的名称上

装饰器本质上是一个**高阶函数**，它接受一个函数作为参数，并返回一个新的函数。主要功能是动态地修改函数或方法的行为，通常用于以下场景：
- 日志记录：在函数执行前后添加日志
- 性能测试：测量函数的执行时间
- 权限检查：在函数执行前验证用户权限
- 缓存：缓存函数的计算结果，避免重复计算
- 追踪：记录函数的调用信息，如参数和返回值